# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#! conda install -c pyviz hvplot geoviews

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key



import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="holoviews")

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zaozersk,69.4005,32.4476,-1.91,88,99,7.16,RU,1701280777
1,1,puerto natales,-51.7236,-72.4875,10.25,53,75,10.80,CL,1701280708
2,2,edinburgh of the seven seas,-37.0676,-12.3116,12.77,72,86,10.88,SH,1701280720
3,3,waitangi,-43.9535,-176.5597,11.01,81,100,3.58,NZ,1701280724
4,4,port lincoln,-34.7333,135.8667,15.42,87,98,6.17,AU,1701280727


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
#%%capture --no-display

Humidity = city_data_df["Humidity"].astype(float)
city = city_data_df["City"]

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = Humidity*100 ,
    scale= 0.07,
    color = city
    )


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
new_dataframe= city_data_df.loc[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] > 21)]
new_dataframe= new_dataframe.loc[new_dataframe["Wind Speed"] < 4.5 ]
new_dataframe= new_dataframe.loc[new_dataframe["Cloudiness"] == 0 ]
# Drop any rows with null values
new_dataframe= new_dataframe.dropna()

# Display sample data
new_dataframe

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,79,taoudenni,22.6783,-3.9836,23.69,12,0,2.84,ML,1701280796
119,119,dakhla,23.6848,-15.9580,22.01,56,0,4.12,EH,1701280820
194,194,atar,20.5169,-13.0499,26.15,11,0,2.47,MR,1701280860
230,230,valabhipur,21.8833,71.8667,21.78,63,0,4.22,IN,1701280886
240,240,tsiombe,-25.3000,45.4833,24.58,86,0,2.25,MG,1701280895
415,415,rouissat,31.9243,5.3502,22.04,18,0,1.54,DZ,1701280980
496,496,aswan,24.0934,32.9070,23.61,25,0,1.54,EG,1701281026
532,532,timimoun,29.2639,0.2310,22.21,13,0,1.03,DZ,1701281037


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=new_dataframe[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
79,taoudenni,ML,22.6783,-3.9836,12,
119,dakhla,EH,23.6848,-15.9580,56,
194,atar,MR,20.5169,-13.0499,11,
230,valabhipur,IN,21.8833,71.8667,63,
240,tsiombe,MG,-25.3000,45.4833,86,
415,rouissat,DZ,31.9243,5.3502,18,
496,aswan,EG,24.0934,32.9070,25,
532,timimoun,DZ,29.2639,0.2310,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel

radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude =hotel_df.loc[index, "Lat"]
    longitude =hotel_df.loc[index, "Lng"]
  
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address_1 = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address_1.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
dakhla - nearest hotel: Hotel El Massira
atar - nearest hotel: فندق سكليل
valabhipur - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
rouissat - nearest hotel: فندق الأروي الذهبي
aswan - nearest hotel: Yaseen hotel
timimoun - nearest hotel: أناف


,City,Country,Lat,Lng,Humidity,Hotel Name
79,taoudenni,ML,22.6783,-3.9836,12,No hotel found
119,dakhla,EH,23.6848,-15.9580,56,Hotel El Massira
194,atar,MR,20.5169,-13.0499,11,فندق سكليل
230,valabhipur,IN,21.8833,71.8667,63,No hotel found
240,tsiombe,MG,-25.3000,45.4833,86,No hotel found
415,rouissat,DZ,31.9243,5.3502,18,فندق الأروي الذهبي
496,aswan,EG,24.0934,32.9070,25,Yaseen hotel
532,timimoun,DZ,29.2639,0.2310,13,أناف


In [16]:
name_address

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'أناف',
    'country': 'Algeria',
    'country_code': 'dz',
    'state': 'Timimoun Province',
    'county': 'Timimoun District',
    'city': 'Timimoun',
    'street': 'شارع أول نوفمبر',
    'lon': 0.2332641,
    'lat': 29.2637913,
    'formatted': 'أناف, شارع أول نوفمبر, Timimoun, Algeria',
    'address_line1': 'أناف',
    'address_line2': 'شارع أول نوفمبر, Timimoun, Algeria',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': ['details'],
    'datasource': {'sourcename': 'openstreetmap',
     'attribution': '© OpenStreetMap contributors',
     'license': 'Open Database Licence',
     'url': 'https://www.openstreetmap.org/copyright',
     'raw': {'name': 'أناف',
      'osm_id': 6592085686,
      'name:ar': 'أناف',
      'name:fr': 'Anaf',
      'tourism': 'hotel',
      'osm_type': 'n',
      'addr:city': 'تيميمون'}},
    'distance': 220,
    'place_id': '51115a6a1899dbcd3f

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
#%%capture --no-display
Humidity = hotel_df["Humidity"].astype(float)
hotel = hotel_df["Hotel Name"]
# Configure the map plot

map_plot_2= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = Humidity*100,
    scale= 0.07,
    color = hotel,
    hover_cols=["City", "Hotel Name", "Country"]
    )



# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size,City,Hotel Name,Country)

In [25]:
#%%capture --no-display
Humidity = hotel_df["Humidity"].astype(float)
hotel = hotel_df["Hotel Name"]
# Configure the map plot

map_plot_2= hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = Humidity*1000,
    scale= 0.07,
    color = hotel,
    hover_cols=["City", "Hotel Name", "Country"]
    )



# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size,City,Hotel Name,Country)